# 数据预处理 得到udf

In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
import math 
import random 


In [2]:
ls = list(pd.read_csv(r"gender_submission.csv").columns) 
for i in list(pd.read_csv(r"test.csv").columns):
    ls.append(i) 
for i in list(pd.read_csv(r"train.csv").columns):
    ls.append(i) 
ls = set(ls) 
ls

{'Age',
 'Cabin',
 'Embarked',
 'Fare',
 'Name',
 'Parch',
 'PassengerId',
 'Pclass',
 'Sex',
 'SibSp',
 'Survived',
 'Ticket',
 'Unnamed: 0'}

In [3]:
# 处理成中文字段
name_dict = {
    "Age": "年龄",
    "Cabin": "船舱号码",
    "Embarked": "登船港口",
    "Fare": "票价",
    "Name": "姓名",
    "Parch": "同船的父母/孩子数量",
    "PassengerId": "乘客ID",
    "Pclass": "舱位等级",
    "Sex": "性别",
    "SibSp": "同船的兄弟姐妹/配偶数量",
    "Survived": "是否生还",
    "Ticket": "船票号码"
}   

In [4]:
# 特证工程:
df1 = pd.read_csv(r"gender_submission.csv") 
df1.columns = df1.columns.map(name_dict)
df2 = pd.read_csv(r"test.csv") 
df2.columns = df2.columns.map(name_dict) 
df3 = pd.read_csv(r"train.csv") 
df3.columns = df3.columns.map(name_dict)

In [5]:
df2

,乘客ID,舱位等级,姓名,性别,年龄,同船的兄弟姐妹/配偶数量,同船的父母/孩子数量,船票号码,票价,船舱号码,登船港口
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [6]:
us_list = ['舱位等级', '性别', '年龄', '同船的兄弟姐妹/配偶数量', '同船的父母/孩子数量', '船票号码', '票价', '船舱号码', '登船港口'] 

#处理数据
def Work_Data(df):
    for i in df.columns:
        us = df[i].mode()[0] 
        df[i].fillna(us, inplace = True)    
        if isinstance(list(df[i])[0], (int, float)):
            continue
        dic = dict() 
        idx = 0 
        for j in set(df[i]): 
            dic[j] = idx
            idx += 1 
        for j in df.index:
            df.loc[j, i] = dic[df.loc[j, i]] 
    return df 


df = Work_Data(df3) 
train_x, train_y = [], [] 

for i in df.index:
    uslist = [] 
    for j in us_list:
        uslist.append(df.loc[i,j]) 
    train_x.append(uslist) 
    train_y.append(df.loc[i,'是否生还'])
    
# 构造测试集合：
test_x = [] 
df = Work_Data(df2) 
for i in df.index:
    uslist = [] 
    for j in us_list:
        uslist.append(df.loc[i,j]) 
    test_x.append(uslist) 


# 训练逻辑回归模型
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


# 特征缩放
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_x)
X_test_scaled = scaler.transform(test_x)

# 初始化逻辑回归模型
log_reg = LogisticRegression()

# 拟合模型
log_reg.fit(X_train_scaled, train_y)


# 预测测试集
y_pred = log_reg.predict(X_test_scaled)


for i in y_pred:
    print(i) 


 

0
0
0
0
1
0
1
0
1
0
0
0
1
0
1
1
0
0
1
1
0
0
1
1
1
0
1
0
0
0
0
0
1
1
1
0
1
1
0
0
0
1
0
1
1
0
0
0
1
1
0
0
1
1
0
0
0
0
0
1
0
0
0
1
1
1
1
0
1
1
1
0
1
1
1
1
0
1
0
1
0
0
0
0
0
0
1
1
1
0
1
0
1
0
1
0
1
0
1
0
1
0
0
0
1
0
0
0
0
0
0
1
1
1
1
0
0
1
1
1
1
0
1
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
1
0
1
0
1
0
0
1
0
0
1
1
0
1
1
0
1
0
0
1
0
0
1
1
0
0
0
0
0
1
1
0
1
1
0
0
1
0
1
0
1
0
0
0
0
1
0
0
0
0
1
1
0
1
1
0
1
1
0
1
1
0
1
0
0
0
0
1
0
0
1
0
1
0
1
0
1
0
1
1
0
1
0
0
0
1
0
0
0
0
0
0
1
1
1
1
0
0
0
0
1
0
1
1
1
0
1
0
0
0
0
0
1
0
0
0
1
1
0
0
1
0
1
0
0
0
1
1
0
1
0
0
0
0
1
0
1
1
1
0
0
0
0
0
1
1
0
0
0
0
1
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
1
0
1
0
0
0
1
1
0
1
0
1
0
0
0
0
0
0
0
1
0
1
0
1
0
1
1
0
0
0
1
0
1
0
0
1
0
1
1
0
1
0
0
1
1
0
0
1
0
0
1
1
1
0
0
0
0
0
1
1
0
1
0
0
0
0
1
1
0
0
0
1
0
1
0
0
1
0
1
1
0
0
0
0
1
1
1
1
1
0
1
0
0
0


In [7]:
df = pd.read_csv(r"gender_submission.csv")
df['Survived'] = y_pred 

df.to_csv(r"gender_submission.csv")